In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import commonmodules as cm

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
filename = "N2H2_2D.xlsx"
excf = pd.ExcelFile(filename)
debug = False
print(excf.sheet_names)

df1 = pd.read_excel(excf, "dv=1")
df2 = pd.read_excel(excf, "dv=2")
df3 = pd.read_excel(excf, "dv=3")

print(df1.columns)
print(df2.columns)
print(df3.columns)


cE = Collision Energy
dE = Delta E 
cS = Cross Section

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = {}
x_s = {}
y = {} 
y_s = {}
scalerx = {}
scalery = {}
vmap_toreal = {}
vset = {}

x["1_v_cE"] = df1[['v', 'cE']].values
x["1_dE_cE"] = df1[['dE', 'cE']].values
y["1"] = np.log10(df1[["cS"]].values)

x["2_v_cE"] = df2[['v', 'cE']].values
x["2_dE_cE"] = df2[['dE', 'cE']].values
y["2"] = np.log10(df2[["cS"]].values)

x["3_v_cE"] = df3[['v', 'cE']].values
x["3_dE_cE"] = df3[['dE', 'cE']].values
y["3"] = np.log10(df3[["cS"]].values)

xkey = ["1_v_cE", "1_dE_cE", \
        "2_v_cE", "2_dE_cE", \
        "3_v_cE", "3_dE_cE"]

ykey = ["1", "2", "3"]

for k in xkey:
    scalerx[k] = MinMaxScaler()
    scalerx[k].fit(x[k])
    x_s[k] = scalerx[k].transform(x[k])

    vmap = {}

    for i, vn in enumerate(x_s[k][:,0]):
        vmap[vn] = x[k][i,0]

    vmap_toreal[k] = vmap
    
    vset[k] = set(x_s[k][:,0])

    if debug:
        for i, xs in enumerate(x_s[k]):
            print(xs, x[k][i])

for k in ykey:
    scalery[k] = MinMaxScaler()
    scalery[k].fit(y[k])
    y_s[k] = scalery[k].transform(y[k])

    if debug:
        for i, ys in enumerate(y_s[k]):
            print(ys, y[k][i])


In [ ]:
from tensorflow import keras
import tensorflow as tf

import tensorflow.keras.optimizers as tko
import tensorflow.keras.activations as tka
import tensorflow.keras.losses as tkl
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
modelshape = [32, 32, 32, 32]
epochs = 20
batch_size = 100

for xk in xkey:
    yk = xk.split("_")[0]
    ofp = open("vremoved_NN"+xk+".csv", "w")

    thefirst = True

    print (" v Removed , Test MSE , Test R2 , Train MSE , Train R2")
    print (" v Removed , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
    for v in vset[xk]:
        train_x, test_x, train_y, test_y = cm.test_train_split (0, [v], x_s[xk], y_s[yk])

        if thefirst:
            model = cm.buildmodel(modelshape, inputshape=2)
            #print(model.summary())
            history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
                verbose=0)
            thefirst = False

        model = cm.buildmodel(modelshape, inputshape=2)
        history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
            verbose=0)
    
   